# Chatbot Example: Arxiv Paper

## Setup Environment

In [8]:
#%pip install arxiv
#%pip install openai
#%pip install python-dotenv

In [13]:
import arxiv
import json
import os
from typing import List
from dotenv import load_dotenv

## Tool Functions

In [14]:
PAPER_DIR = "./../papers"

In [15]:
from typing import List
import os
import json
import arxiv


def search_papers(topic: str, max_results: int = 5) -> List[str]:
    """
    주제를 기반으로 arXiv에서 논문을 검색하고 해당 정보를 저장한다.

    Args:
        topic: 검색할 주제
        max_results: 검색할 최대 결과 수 (기본값: 5)

    Returns:
        검색에서 찾은 논문 ID 목록
    """
    # arxiv를 사용하여 논문 찾기
    client = arxiv.Client()
    search = arxiv.Search(
        query     = topic,
        max_results = max_results,
        sort_by   = arxiv.SortCriterion.Relevance
    )
    papers = client.results(search)

    # 이 주제에 대한 디렉토리 생성
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    os.makedirs(path, exist_ok=True)

    file_path = os.path.join(path, "papers_info.json")

    # 기존 논문 정보 로드 시도
    try:
        with open(file_path, "r") as json_file:
            papers_info = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}

    # 각 논문을 처리하고 papers_info에 추가
    paper_ids = []
    for paper in papers:
        paper_id   = paper.get_short_id()
        paper_info = {
            "title"    : paper.title,
            "authors"  : [author.name for author in paper.authors],
            "summary"  : paper.summary,
            "pdf_url"  : paper.pdf_url,
            "published": str(paper.published.date())
        }
        paper_ids.append(paper_id)
        papers_info[paper_id] = paper_info

    # 업데이트된 papers_info를 json 파일에 저장
    with open(file_path, "w") as json_file:
        json.dump(papers_info, json_file, indent=2)

    print(f"결과가 다음 위치에 저장되었습니다: {file_path}")
    return paper_ids

In [16]:
search_papers("computers")

결과가 다음 위치에 저장되었습니다: ./../papers/computers/papers_info.json


['1310.7911v2',
 'math/9711204v1',
 '2208.00733v1',
 '2504.07020v1',
 '2403.03925v1']

In [17]:
def extract_info(paper_id: str) -> str:
    """
    모든 주제 디렉토리에서 특정 논문에 관한 정보를 검색한다.
    
    Args:
        paper_id: 검색할 논문의 ID
        
    Returns:
        찾은 경우 논문 정보가 담긴 JSON 문자열, 찾지 못한 경우 오류 메시지
    """
 
    for item in os.listdir(PAPER_DIR):
        item_path = os.path.join(PAPER_DIR, item)
        if os.path.isdir(item_path):
            file_path = os.path.join(item_path, "papers_info.json")
            if os.path.isfile(file_path):
                try:
                    with open(file_path, "r") as json_file:
                        papers_info = json.load(json_file)
                        if paper_id in papers_info:
                            return json.dumps(papers_info[paper_id], indent=2)
                except (FileNotFoundError, json.JSONDecodeError) as e:
                    print(f"{file_path} 읽기 오류: {str(e)}")
                    continue
    
    return f"논문 {paper_id}와 관련된 저장된 정보가 없다."

In [18]:
extract_info('1310.7911v2')

'{\n  "title": "Compact manifolds with computable boundaries",\n  "authors": [\n    "Zvonko Iljazovic"\n  ],\n  "summary": "We investigate conditions under which a co-computably enumerable closed set\\nin a computable metric space is computable and prove that in each locally\\ncomputable computable metric space each co-computably enumerable compact\\nmanifold with computable boundary is computable. In fact, we examine the notion\\nof a semi-computable compact set and we prove a more general result: in any\\ncomputable metric space each semi-computable compact manifold with computable\\nboundary is computable. In particular, each semi-computable compact\\n(boundaryless) manifold is computable.",\n  "pdf_url": "http://arxiv.org/pdf/1310.7911v2",\n  "published": "2013-10-29"\n}'

## Tool Schema

In [19]:
functions = [
    {
        "name": "search_papers",
        "description": "주제를 기반으로 arXiv에서 논문을 검색하고 해당 정보를 저장한다.",
        "parameters": {
            "type": "object",
            "properties": {
                "topic": {
                    "type": "string",
                    "description": "검색할 주제"
                },
                "max_results": {
                    "type": "integer",
                    "description": "검색할 최대 결과 수",
                    "default": 5
                }
            },
            "required": ["topic"]
        }
    },
    {
        "name": "extract_info",
        "description": "모든 주제 디렉토리에서 특정 논문에 관한 정보를 검색한다.",
        "parameters": {
            "type": "object",
            "properties": {
                "paper_id": {
                    "type": "string",
                    "description": "검색할 논문의 ID"
                }
            },
            "required": ["paper_id"]
        }
    }
]

## Tool Mapping

In [20]:
mapping_tool_function = {
    "search_papers": search_papers,
    "extract_info": extract_info
}

def execute_tool(tool_name, tool_args):
    result = mapping_tool_function[tool_name](**tool_args)
    if result is None:
        return "작업이 완료되었지만 결과를 반환하지 않았다."
    
    if isinstance(result, list):
        return ", ".join(result)
    
    if isinstance(result, dict):
        return json.dumps(result, indent=2, ensure_ascii=False)
    
    return str(result)

## Chatbot Code

In [22]:
from openai import OpenAI
from dotenv import load_dotenv
import os

#!echo "<당신의 OpenAI Key를 여기에 붙혀넣으세요>" >> .env
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

python-dotenv could not parse statement starting at line 2


### Query Processing

In [23]:
def process_query(query: str):
    messages = [{"role": "user", "content": query}]

    while True:
        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            functions=functions,
            function_call="auto",
            max_tokens=2024
        )
        msg = resp.choices[0].message

        # 1) 일반 텍스트 응답
        if msg.content is not None:
            print(msg.content)
            break

        # 2) 함수 호출 응답
        call = msg.function_call
        fname = call.name
        fargs = json.loads(call.arguments)

        print(f"Calling tool `{fname}` with args: {fargs}")
        tool_result = execute_tool(fname, fargs)

        # assistant의 함수 호출 메시지를 기록
        messages.append({
            "role": "assistant",
            "function_call": {
                "name": fname,
                "arguments": call.arguments
            }
        })
        # 함수 실행 결과를 function 역할로 추가
        messages.append({
            "role": "function",
            "name": fname,
            "content": tool_result
        })
        # 루프 계속 → 모델이 function 결과를 바탕으로 응답 생성

### Chat Loop

In [24]:
def chat_loop():
    print("쿼리를 입력하거나 'quit'를 입력해 종료합니다.")
    while True:
        query = input("Query: ").strip()
        if query.lower() == "quit":
            break
        try:
            process_query(query)
        
        except Exception as e:
            print(f"Error: {e}")

In [25]:
chat_loop()

쿼리를 입력하거나 'quit'를 입력해 종료합니다.


Query:  논문 제목을 알려줄래


어떤 주제에 대한 논문 제목을 알고 싶으신가요? 특정한 분야나 키워드를 말씀해 주시면 관련 논문을 찾아드릴 수 있습니다.


Query:  LLM 해석 가능성에 대한 논문 2편을 검색해줘.


Calling tool `search_papers` with args: {'topic': 'LLM interpretability', 'max_results': 2}
결과가 다음 위치에 저장되었습니다: ./../papers/llm_interpretability/papers_info.json
Calling tool `extract_info` with args: {'paper_id': '2412.07992v3'}
Calling tool `extract_info` with args: {'paper_id': '2402.01761v1'}
다음은 LLM 해석 가능성에 관한 두 편의 논문입니다.

1. **논문 제목**: [Concept Bottleneck Large Language Models](http://arxiv.org/pdf/2412.07992v3)  
   **저자**: Chung-En Sun, Tuomas Oikarinen, Berk Ustun, Tsui-Wei Weng  
   **요약**: 이 논문에서는 본질적으로 해석 가능한 LLM을 구축하기 위한 새로운 프레임워크인 개념 병목 LLM (CB-LLMs)을 소개합니다. CB-LLMs는 전통적인 블랙 박스 LLM과 달리 LLM에 본질적인 해석 가능성을 통합하여 정확한 설명을 제공합니다. 텍스트 분류와 생성이라는 두 가지 핵심 NLP 작업에 대해 CB-LLMs를 구축하며, 경쟁력 있는 성능을 보여주고 해석 가능한 이유를 제공합니다. 이 모델은 유해 콘텐츠 식별, 모델 행동 조정 및 원치 않는 개념 언learn 능력을 강화하여 LLM의 안전성과 신뢰성을 크게 향상시킵니다.  
   **발행 일자**: 2024-12-11

2. **논문 제목**: [Rethinking Interpretability in the Era of Large Language Models](http://arxiv.org/pdf/2402.01761v1)  
   **저자**: Chandan Singh, Jeevana Priya Inala, Mi

Query:  quit


## OpenAI 전체 코드

In [28]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 1) OpenAI 함수 호출 형식으로 도구 정의
functions = [
    {
        "name": "search_papers",
        "description": "주제를 기반으로 arXiv에서 논문을 검색하고 해당 정보를 저장한다.",
        "parameters": {
            "type": "object",
            "properties": {
                "topic": {
                    "type": "string",
                    "description": "검색할 주제"
                },
                "max_results": {
                    "type": "integer",
                    "description": "검색할 최대 결과 수",
                    "default": 5
                }
            },
            "required": ["topic"]
        }
    },
    {
        "name": "extract_info",
        "description": "모든 주제 디렉토리에서 특정 논문에 관한 정보를 검색한다.",
        "parameters": {
            "type": "object",
            "properties": {
                "paper_id": {
                    "type": "string",
                    "description": "검색할 논문의 ID"
                }
            },
            "required": ["paper_id"]
        }
    }
]

# 2) 실제 함수 매핑
mapping_tool_function = {
    "search_papers": search_papers,
    "extract_info": extract_info
}

def execute_tool(tool_name, tool_args):
    result = mapping_tool_function[tool_name](**tool_args)
    if result is None:
        return "작업이 완료되었지만 결과를 반환하지 않았다."
    if isinstance(result, list):
        return ", ".join(result)
    if isinstance(result, dict):
        return json.dumps(result, indent=2, ensure_ascii=False)
    return str(result)

def process_query(query: str):
    messages = [{"role": "user", "content": query}]

    while True:
        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            functions=functions,
            function_call="auto",
            max_tokens=2024
        )
        msg = resp.choices[0].message

        # 1) 일반 텍스트 응답
        if msg.content is not None:
            print(msg.content)
            break

        # 2) 함수 호출 응답
        call = msg.function_call
        fname = call.name
        fargs = json.loads(call.arguments)

        print(f"Calling tool `{fname}` with args: {fargs}")
        tool_result = execute_tool(fname, fargs)

        # assistant의 함수 호출 메시지를 기록
        messages.append({
            "role": "assistant",
            "function_call": {
                "name": fname,
                "arguments": call.arguments
            }
        })
        # 함수 실행 결과를 function 역할로 추가
        messages.append({
            "role": "function",
            "name": fname,
            "content": tool_result
        })
        # 루프 계속 → 모델이 function 결과를 바탕으로 응답 생성

def chat_loop():
    print("쿼리를 입력하거나 'quit'를 입력해 종료합니다.")
    while True:
        query = input("Query: ").strip()
        if query.lower() == "quit":
            break
        try:
            process_query(query)
        except Exception as e:
            print(f"Error: {e}")

if __name__ == "__main__":
    chat_loop()

쿼리를 입력하거나 'quit'를 입력해 종료합니다.


Query:  논문 제목을 알려줄래


원하는 논문 제목이나 주제가 있으신가요? 특정 주제나 분야에 대한 논문 제목을 찾고 싶으시면 말씀해 주세요.


Query:  quit
